# Basic Ollama API to Summarizing a Website

In [2]:
import os
import re
import requests
import time
import json
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [3]:
# Class to represent a website

# Headers for some websites that block default user agents
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class Website:
    """
    A utility class to represent the website we scraped
    """
    url:str
    title:str
    text:str

    def __init__(self, url):
        """
        Create this website object from the given url using BeautifulSoup library
        """
        self.url = url
        response = requests.get(url=url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        print(self.title)
        for irrelevent in soup.body.find_all(["script", "style", "img", "input", "noscript"]):
            irrelevent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
test_url = Website("https://en.wikipedia.org/wiki/Free-software_license")

Free-software license - Wikipedia


In [5]:
print(test_url.title)

Free-software license - Wikipedia


In [6]:
print(test_url.text[:100])

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current event


### Types of Prompt

**A system prompt** -- that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary within 500 characters, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes user prompt for the summarization of the websites

def create_user_prompt(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(create_user_prompt(test_url)[:1000])

You are looking at a website titled Free-software license - Wikipedia
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Special pages
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Contents
move to sidebar
hide
(Top)
1
Comparison
2
History
Toggle History subsection
2.1
Pre-1980s
2.2
1980s
2.3
1990s to 2000s
2.4
2010s
3
Definitions
Toggle Definitions subsection
3.1
OSI-approved open-source licenses
3.2
FSF-approved free-software licenses
4
Conditions in free-software licenses
Toggle Conditions in free-software licenses subsection
4.1
Copyleft
4.2
Patent retaliation
4.3
Attribution, disclaimers and notices


### Messages
All Ollama APIs share this structure:
```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```

In [10]:
# Creating messages for Ollama

def messages_ollama(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": create_user_prompt(website)}
    ]

In [11]:
# print(messages_ollama(test_url))

In [17]:
MODEL_NAME="qwen3"  # Get this from you local ollama by using "ollama list"
OLLAMA_URL = "http://localhost:11434/api/chat"

In [18]:
# Calling the Ollama APIs

def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL_NAME,
        "messages": messages_ollama(website),
        "stream": False  # Important: to get streaming responses
    }
    start_time = time.time()
    response = requests.post(OLLAMA_URL, json=payload, stream=False)
    print("Total Time Taken: ", time.time()-start_time)
    reply = response.json()["message"]["content"]
    reply = re.sub(r"<think>.*?</think>", "", reply, flags=re.DOTALL).strip()
    return reply
    

In [19]:
# summarize("https://en.wikipedia.org/wiki/Free_and_open-source_software")

In [20]:
# A function to display the summary in Markdown un jupyter

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://en.wikipedia.org/wiki/Free_and_open-source_software")

Free and open-source software - Wikipedia
Total Time Taken:  406.6380536556244


The provided text is the content of a Wikipedia article titled **"Free and open-source software"**, which explains the concept, history, licenses, and significance of free and open-source software (FOSS). Below is a **concise summary** and **key highlights** of the topic:

---

### **What is Free and Open-Source Software (FOSS)?**
FOSS refers to software that is **freely available** (in terms of freedom, not necessarily price) and whose **source code is openly shared**. Users can **use, modify, and distribute** the software under licenses that ensure these freedoms.

---

### **Key Characteristics of FOSS**
1. **Four Essential Freedoms** (as defined by the Free Software Foundation):
   - **Freedom to use** the software for any purpose.
   - **Freedom to study** and modify the source code.
   - **Freedom to redistribute** copies (either gratis or for a fee).
   - **Freedom to improve** the software and release improvements to the public.

2. **Open Source**:
   - Emphasizes **collaborative development** and **transparency**.
   - Focuses on **practical benefits** (e.g., cost savings, innovation) rather than philosophical freedom.

---

### **Popular FOSS Licenses**
- **GPL (GNU General Public License)**: Ensures software remains free and open.
- **MIT License**: Permissive, allowing modification and redistribution with minimal restrictions.
- **Apache License 2.0**: Includes patent grants and allows proprietary derivatives.
- **BSD License**: Simple and permissive, often used in academic and research projects.
- **Creative Commons (CC)**: For non-software works (e.g., media, data).

---

### **Notable FOSS Projects**
- **Linux**: An open-source operating system kernel.
- **Apache HTTP Server**: A widely used web server.
- **Mozilla Firefox**: An open-source web browser.
- **WordPress**: A content management system (CMS).
- **LibreOffice**: A free office suite.
- **GIMP**: A raster graphics editor.
- **Python**: A popular programming language with an open-source license.

---

### **History and Impact**
- **1980s–1990s**: Richard Stallman founded the **Free Software Foundation (FSF)** to promote free software.
- **1998**: The **Open Source Initiative (OSI)** was created to promote open-source software, distinguishing it from free software philosophically.
- **Modern Era**: FOSS drives innovation in technology, education, and research. It underpins much of the internet (e.g., Linux, Apache, MySQL).

---

### **Challenges and Criticisms**
- **License Proliferation**: Over 100 licenses exist, complicating compliance.
- **Security Risks**: Open-source software can be vulnerable if not properly maintained.
- **Commercialization**: Some companies use FOSS for profit, raising debates about "open-source" vs. "free software."

---

### **FOSS vs. Proprietary Software**
| **Aspect**               | **FOSS**                          | **Proprietary Software**         |
|--------------------------|-----------------------------------|-----------------------------------|
| **Source Code**          | Open and modifiable               | Closed, proprietary               |
| **Cost**                 | Often free (gratis) or low-cost  | Paid (may include subscription)   |
| **Customization**        | High                              | Limited (unless purchased)        |
| **Community Involvement**| Encouraged                        | Typically limited to developers   |

---

### **Significance in Modern Technology**
- **Innovation**: FOSS fosters collaboration and rapid development (e.g., Kubernetes, TensorFlow).
- **Education**: Tools like Python, R, and LaTeX are staples in academic research.
- **Global Access**: Provides affordable solutions for underserved communities.
- **Ethical and Philosophical**: Promotes transparency, equity, and resistance to monopolies.

---

### **Key Organizations**
- **Free Software Foundation (FSF)**: Advocates for free software.
- **Open Source Initiative (OSI)**: Promotes open-source software.
- **Apache Software Foundation (ASF)**: Manages open-source projects like Apache HTTP Server.
- **Linux Foundation**: Supports Linux and other open-source initiatives.

---

### **Further Reading**
- **"Free as in Freedom" by Sam Williams**: Explores Richard Stallman's philosophy.
- **"The Cathedral and the Bazaar" by Eric S. Raymond**: A foundational text on open-source development.
- **"The Success of Open Source" by Steven Weber**: Analyzes the economic and social impact of FOSS.

---

This summary captures the essence of the Wikipedia article, highlighting the **philosophy, licenses, projects, and impact** of free and open-source software. Let me know if you'd like a deeper dive into any specific aspect!

## Introducing the ollama package
And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [22]:
# !pip install ollama

In [23]:
import ollama

In [27]:
response = ollama.chat(model=MODEL_NAME, messages=messages_ollama(test_url))
response["message"]["content"]

'A **free-software license** is a type of license that grants users the **four essential freedoms** defined by the Free Software Foundation (FSF):\n\n1. **Freedom to run the program for any purpose.**  \n2. **Freedom to study how the program works and modify it.**  \n3. **Freedom to redistribute copies.**  \n4. **Freedom to distribute modified versions.**  \n\nThese licenses ensure that software remains **open, modifiable, and redistributable** without restrictions, aligning with the principles of free software. Here’s a breakdown of key concepts and examples:\n\n---\n\n### **Key Characteristics of Free-Software Licenses**\n- **Copyleft vs. Permissive Licenses**:  \n  - **Copyleft licenses** (e.g., GNU GPL) require derivative works to be distributed under the same license, ensuring ongoing openness.  \n  - **Permissive licenses** (e.g., MIT, BSD) allow broader use, including in proprietary software, as long as the original license is included.\n\n- **Enforcement of Freedoms**:  \n  - C

In [28]:
display(Markdown(response["message"]["content"]))

A **free-software license** is a type of license that grants users the **four essential freedoms** defined by the Free Software Foundation (FSF):

1. **Freedom to run the program for any purpose.**  
2. **Freedom to study how the program works and modify it.**  
3. **Freedom to redistribute copies.**  
4. **Freedom to distribute modified versions.**  

These licenses ensure that software remains **open, modifiable, and redistributable** without restrictions, aligning with the principles of free software. Here’s a breakdown of key concepts and examples:

---

### **Key Characteristics of Free-Software Licenses**
- **Copyleft vs. Permissive Licenses**:  
  - **Copyleft licenses** (e.g., GNU GPL) require derivative works to be distributed under the same license, ensuring ongoing openness.  
  - **Permissive licenses** (e.g., MIT, BSD) allow broader use, including in proprietary software, as long as the original license is included.

- **Enforcement of Freedoms**:  
  - Copyleft licenses enforce the four freedoms by requiring source code availability for derivatives.  
  - Permissive licenses prioritize flexibility but may not guarantee the same level of user freedom.

---

### **Examples of Free-Software Licenses**
1. **GNU General Public License (GPL)**  
   - A **copyleft** license.  
   - Derivative works must also be licensed under GPL.  
   - Popular for Linux, GNU tools, and many open-source projects.

2. **MIT License**  
   - A **permissive** license.  
   - Allows use in proprietary software, with minimal requirements (e.g., attribution).  
   - Widely used for libraries and APIs.

3. **Apache License 2.0**  
   - Permissive, with additional clauses for patent grants and compatibility with other licenses.  
   - Common in cloud and enterprise software.

4. **BSD Licenses**  
   - Variants include 2-clause (BSD) and 3-clause (Modified BSD).  
   - Permissive, with minimal restrictions (e.g., no warranty).

5. **GNU Affero General Public License (AGPL)**  
   - A copyleft license designed for web services.  
   - Ensures users interacting with the software over a network must receive the source code.

---

### **Differences Between Free Software and Open Source**
- **Free Software (FSF)**: Emphasizes **user freedoms** (four freedoms) and ethical principles.  
- **Open Source (OSI)**: Focuses on **practicality** (e.g., source code availability, modifiability) without strict philosophical requirements.  
- **Overlap**: Many open-source licenses (e.g., GPL, MIT) are also free software licenses, but not all open-source licenses meet the FSF’s criteria.

---

### **Legal and Philosophical Considerations**
- **Copyleft** (e.g., GPL): Prevents "free riding" by requiring derivatives to remain open.  
- **Permissive Licenses**: Allow broader commercial use but may not enforce long-term openness.  
- **Creative Commons (CC)**: Not typically considered free software licenses, as they may include restrictions (e.g., non-commercial use).  
- **Unlicense**: A public domain dedication, but not widely used due to legal uncertainties.

---

### **Challenges and Concerns**
- **Commercial Use**: Permissive licenses may allow proprietary software to incorporate free software without contributing back.  
- **License Proliferation**: Over 100 licenses exist, leading to complexity in compliance.  
- **FSF vs. OSI**: The FSF prioritizes user freedoms, while the OSI focuses on practical usability.

---

### **Conclusion**
Free-software licenses are critical for maintaining the **openness and freedom** of software. Copyleft licenses like the GPL ensure that derivatives remain free, while permissive licenses like MIT and BSD offer flexibility. Choosing the right license depends on the project’s goals—whether to enforce strict openness or allow broader use. Understanding these licenses is essential for developers, users, and organizations seeking to collaborate in the free software ecosystem.